In [2]:
import pandas as pd
import numpy as np
import seaborn as sns;
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [3]:
# loading the data into the dataframe
df = pd.read_csv('data.csv') 
print(df) 

          No  year  month  day  hour  PM2.5  PM10   SO2   NO2   CO    O3  \
0          1  2013      3    1     0    4.0   4.0   4.0   7.0  300  77.0   
1          2  2013      3    1     1    8.0   8.0   4.0   7.0  300  77.0   
2          3  2013      3    1     2    7.0   7.0   5.0  10.0  300  73.0   
3          4  2013      3    1     3    6.0   6.0  11.0  11.0  300  72.0   
4          5  2013      3    1     4    3.0   3.0  12.0  12.0  300  72.0   
...      ...   ...    ...  ...   ...    ...   ...   ...   ...  ...   ...   
31891  35060  2017      2   28    19   12.0  29.0   5.0  35.0  400  95.0   
31892  35061  2017      2   28    20   13.0  37.0   7.0  45.0  500  81.0   
31893  35062  2017      2   28    21   16.0  37.0  10.0  66.0  700  58.0   
31894  35063  2017      2   28    22   21.0  44.0  12.0  87.0  700  35.0   
31895  35064  2017      2   28    23   19.0  31.0  10.0  79.0  600  42.0   

       TEMP    PRES  DEWP  RAIN   wd  WSPM  
0      -0.7  1023.0 -18.8   0.0  NNW   4.4

In [5]:
# viewing info about the columns
df.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31896 entries, 0 to 31895
Data columns (total 17 columns):
No       31896 non-null int64
year     31896 non-null int64
month    31896 non-null int64
day      31896 non-null int64
hour     31896 non-null int64
PM2.5    31896 non-null float64
PM10     31896 non-null float64
SO2      31896 non-null float64
NO2      31896 non-null float64
CO       31896 non-null int64
O3       31896 non-null float64
TEMP     31876 non-null float64
PRES     31876 non-null float64
DEWP     31876 non-null float64
RAIN     31876 non-null float64
wd       31821 non-null object
WSPM     31882 non-null float64
dtypes: float64(10), int64(6), object(1)
memory usage: 4.1+ MB


In [6]:
#viewing few rows from the top
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300,77.0,-1.1,1023.2,-18.2,0.0,N,4.7
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300,72.0,-2.0,1025.2,-19.5,0.0,N,2.0


In [7]:
#number of rows and columns in the dataset
print(df.shape)

(31896, 17)


In [8]:
#statistical information about columns
print(df.describe())

                 No          year         month           day          hour  \
count  31896.000000  31896.000000  31896.000000  31896.000000  31896.000000   
mean   17933.567595   2014.718930      6.400176     15.584587     11.623276   
std    10071.057418      1.164307      3.419374      8.793479      6.930351   
min        1.000000   2013.000000      1.000000      1.000000      0.000000   
25%     9436.750000   2014.000000      3.000000      8.000000      6.000000   
50%    18132.500000   2015.000000      6.000000     15.000000     12.000000   
75%    26648.250000   2016.000000      9.000000     23.000000     18.000000   
max    35064.000000   2017.000000     12.000000     31.000000     23.000000   

              PM2.5          PM10           SO2           NO2            CO  \
count  31896.000000  31896.000000  31896.000000  31896.000000  31896.000000   
mean      81.885265    109.126806     17.155964     59.017494   1257.362208   
std       80.444972     93.686186     22.743912    

In [9]:
#checking how many null values are in each column
df.isnull().sum()

No        0
year      0
month     0
day       0
hour      0
PM2.5     0
PM10      0
SO2       0
NO2       0
CO        0
O3        0
TEMP     20
PRES     20
DEWP     20
RAIN     20
wd       75
WSPM     14
dtype: int64

In [10]:
# dropping all the rows with NaN values
df.dropna()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300,77.0,-1.1,1023.2,-18.2,0.0,N,4.7
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300,72.0,-2.0,1025.2,-19.5,0.0,N,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31891,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400,95.0,12.5,1013.5,-16.2,0.0,NW,2.4
31892,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500,81.0,11.6,1013.6,-15.1,0.0,WNW,0.9
31893,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700,58.0,10.8,1014.2,-13.3,0.0,NW,1.1
31894,35063,2017,2,28,22,21.0,44.0,12.0,87.0,700,35.0,10.5,1014.4,-12.9,0.0,NNW,1.2


In [11]:
#defining training and testing data
x_train = df[:30525]
y_train = x_train['PM2.5']
x_test = df[30525:31898]
y_test = x_test['PM2.5']
print(y_test)

30525    477.0
30526    454.0
30527    418.0
30528    438.0
30529    453.0
         ...  
31891     12.0
31892     13.0
31893     16.0
31894     21.0
31895     19.0
Name: PM2.5, Length: 1371, dtype: float64


In [12]:
#Normalize training data
train_norm = x_train['PM2.5'] 
train_norm_arr = np.asarray(train_norm)
train_norm = np.reshape(train_norm_arr, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
train_norm = scaler.fit_transform(train_norm)
for i in range(5):
    print(train_norm[i])

[0.00140845]
[0.00704225]
[0.0056338]
[0.00422535]
[0.]


In [13]:
test_norm = x_test['PM2.5']
test_norm_arr = np.asarray(test_norm)
test_norm = np.reshape(test_norm_arr, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
test_norm = scaler.fit_transform(test_norm)
for i in range(5):
    print(test_norm[i])

[0.66760563]
[0.63521127]
[0.58450704]
[0.61267606]
[0.63380282]


In [14]:
print(train_norm.shape)
print(test_norm.shape)

(30525, 1)
(1371, 1)


In [15]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X),array(y)

In [16]:
n_steps = 3
X_split_train, y_split_train = split_sequence(train_norm, n_steps)
#for i in range(len(X_split_train)):
    #print(X_split_train[i], y_split_train[i])
n_features = 1
X_split_train = X_split_train.reshape((X_split_train.shape[0], X_split_train.shape[1], n_features))
for i in range(5):
    print(X_split_train)

[[[0.00140845]
  [0.00704225]
  [0.0056338 ]]

 [[0.00704225]
  [0.0056338 ]
  [0.00422535]]

 [[0.0056338 ]
  [0.00422535]
  [0.        ]]

 ...

 [[0.53239437]
  [0.53802817]
  [0.57183099]]

 [[0.53802817]
  [0.57183099]
  [0.63098592]]

 [[0.57183099]
  [0.63098592]
  [0.66056338]]]
[[[0.00140845]
  [0.00704225]
  [0.0056338 ]]

 [[0.00704225]
  [0.0056338 ]
  [0.00422535]]

 [[0.0056338 ]
  [0.00422535]
  [0.        ]]

 ...

 [[0.53239437]
  [0.53802817]
  [0.57183099]]

 [[0.53802817]
  [0.57183099]
  [0.63098592]]

 [[0.57183099]
  [0.63098592]
  [0.66056338]]]
[[[0.00140845]
  [0.00704225]
  [0.0056338 ]]

 [[0.00704225]
  [0.0056338 ]
  [0.00422535]]

 [[0.0056338 ]
  [0.00422535]
  [0.        ]]

 ...

 [[0.53239437]
  [0.53802817]
  [0.57183099]]

 [[0.53802817]
  [0.57183099]
  [0.63098592]]

 [[0.57183099]
  [0.63098592]
  [0.66056338]]]
[[[0.00140845]
  [0.00704225]
  [0.0056338 ]]

 [[0.00704225]
  [0.0056338 ]
  [0.00422535]]

 [[0.0056338 ]
  [0.00422535]
  [0.       

In [17]:
X_split_test, y_split_test = split_sequence(test_norm, n_steps)
for i in range(5):
    print(X_split_test[i], y_split_test[i])
n_features = 1
X_split_test = X_split_test.reshape((X_split_test.shape[0], X_split_test.shape[1], n_features))

[[0.66760563]
 [0.63521127]
 [0.58450704]] [0.61267606]
[[0.63521127]
 [0.58450704]
 [0.61267606]] [0.63380282]
[[0.58450704]
 [0.61267606]
 [0.63380282]] [0.6056338]
[[0.61267606]
 [0.63380282]
 [0.6056338 ]] [0.58169014]
[[0.63380282]
 [0.6056338 ]
 [0.58169014]] [0.58873239]


In [18]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
#sgd = optimizers.SGD(lr=0.001, decay=1e-5, momentum=1.0, nesterov=False)
sgd = optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True) #good
#keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9)
keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

In [ ]:
# fit model
hist = model.fit(X_split_train, y_split_train, validation_data=(X_split_test, y_split_test), epochs=10, verbose = 1)

In [ ]:
print(hist.history.keys())

In [ ]:
# input = train_norm[33645:33648]
# print(input)
# input_T = input.T
# print(input_T)
yhat = model.predict(X_split_test)
for i in range(5):
    print(yhat[i])

In [ ]:
mse = mean_squared_error(y_split_test, yhat)
print('MSE: %.5f' % mse)

In [ ]:
plt.plot(yhat)

In [ ]:
plt.plot(y_split_test)

In [ ]:
_, train_acc = model.evaluate(X_split_train, y_split_train, verbose=0)
_, test_acc = model.evaluate(X_split_test, y_split_test, verbose=0)
print('Train: %.5f, Test: %.5f' % (train_acc, test_acc))

In [ ]:
# summarize history for accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def compute(var):
    train_norm = x_train[var] 
    train_norm_arr = np.asarray(train_norm)
    train_norm = np.reshape(train_norm_arr, (-1, 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_norm = scaler.fit_transform(train_norm)

    test_norm = x_test[var]
    test_norm_arr = np.asarray(test_norm)
    test_norm = np.reshape(test_norm_arr, (-1, 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    test_norm = scaler.fit_transform(test_norm)

    X_split_train, y_split_train = split_sequence(train_norm, n_steps)
    X_split_train = X_split_train.reshape((X_split_train.shape[0], X_split_train.shape[1], n_features))

    X_split_test, y_split_test = split_sequence(test_norm, n_steps)
    X_split_test = X_split_test.reshape((X_split_test.shape[0], X_split_test.shape[1], n_features))

    hist = model.fit(X_split_train, y_split_train, validation_data=(X_split_test, y_split_test), epochs=10, verbose = 1)

    yhat = model.predict(X_split_test)

    mse = mean_squared_error(y_split_test, yhat)
    print(mse)
    
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
# compute('PM2.5')
# compute('PM10')
# compute('SO2')
# compute('NO2')
# compute('CO')
# compute('O3')

In [ ]:
sns.jointplot(x=df['PM2.5'], y=df['PM10'], data = df)

In [ ]:
corrmat = df.corr()
fig, ax = plt.subplots(figsize=(11,11))
sns.heatmap(corrmat)

In [ ]:
g = sns.pairplot(df)

In [ ]:
df.plot(kind='density', subplots=True, layout=(4,4), sharex=False, figsize=(10,10))
plt.show()

In [ ]:
df.plot.scatter(x='PM2.5', y='PM10', c='DarkBlue')

In [ ]:
plt.scatter(y_split_test, yhat)

In [ ]:
df.plot.scatter(x='PM10', y='SO2', c='DarkBlue')

In [ ]:
df.plot.scatter(x='SO2', y='NO2', c='DarkBlue')

In [ ]:
df.plot.scatter(x='NO2', y='CO', c='DarkBlue')

In [ ]:
df.plot.scatter(x='CO', y='O3', c='DarkBlue')

In [ ]:
correlations = df.corr()
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f', square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
plt.show();